# Homework 1

## Imports

In [1]:
import nbtools
nbtools.setup_nb()

In [2]:
from astropy import units, constants
import numpy
from plotly import graph_objects as go

# Homework code
from gr2 import tov

# Problem C

Common scaling constant $\rho_*$

In [3]:
rho_star = constants.m_n * constants.c ** 2 / ((1.0 * units.fm).to(units.m) ** 3)
rho_star / constants.M_sun

<Quantity 75706.2105281 1 / (m s2)>

## Problem C.1

### Establish EoS

In [4]:
gamma = 2.0
alpha = 0.5

n = 1 / (gamma - 1)
K_bar = 1.0 # used for simulation
K = alpha / (rho_star ** (gamma - 1)) # used for scaling results

In [22]:
K

<Quantity 3.32148722e-36 m s2 / kg>

### Compute TOV solutions for various rest densities

Set densities

In [23]:
mid_cent_density = 0.5 * rho_star
mid_cent_density_bar = (K ** n) * mid_cent_density

In [24]:
mid_cent_density_bar

<Quantity 0.25>

In [25]:
densities_rest = numpy.arange(0.1, 1.5, 0.1)

d_rad = 0.001
rad_max = 2.0
solver = tov.integrate_manual # method used by Prof. Radice in NR

Compute solutions

In [26]:
seq = tov.SolutionSequence.from_densities(densities_rest=densities_rest, d_rad=d_rad, 
                                          poly_index=n, poly_gas_const=K_bar, 
                                          rad_max=rad_max, solver=solver)

### Compute TOV Solution for $\rho_c=0.5 \rho_*$

In [52]:
cent_density = 0.5 * rho_star
cent_density_bar = (K ** n) * mid_cent_density

# mid_cent_density_rest = tov.polytropic_density_rest(tov.polytropic_pressure(mid_cent_density))

In [53]:
densities_rest = [cent_density_bar]

d_rad = 0.001
rad_max = 2.0
solver = tov.integrate_manual # method used by Prof. Radice in NR

In [54]:
_seq = tov.SolutionSequence.from_densities(densities_rest=densities_rest, d_rad=d_rad, 
                                           poly_index=n, poly_gas_const=K_bar, 
                                           rad_max=rad_max, solver=solver)
soln = _seq.solutions[0]

In [55]:
soln.surface_radius

0.8106050288104911

In [ ]:
constants.r

In [ ]:
soln.surface_radius * (8 / numpy.pi) * constants.c / 1000 / numpy.sqrt(constants.G)

In [56]:
tov.scale_radius(soln.surface_radius, n, K)

<Quantity 1.47732343e-18 m(1/2) s / kg(1/2)>

### Visualize Results

In [27]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=seq.densities_rest, 
                         y=tov.scale_radius(seq.surface_radius, n, K)))
fig.update_layout(
#     yaxis_range=[-5,5],
#     xaxis_range=[0,3],
                  width=800,
                  height=600,
#                   showlegend=True,
                  title_text=r'$\text{Radius v. Rest-Density }$', 
                  title_x=0.5,
                  xaxis_title=r'$\rho_0 / \rho_*$',
                  yaxis_title=r'$R$')
fig.show()

In [20]:
K

<Quantity 3.32148722e-36 m s2 / kg>

In [30]:
from scipy import optimize

In [47]:
def polytropic_density_rest(density, index, gas_const):
    def func(density_rest):
        return density - tov.polytropic_density(density_rest=density_rest, 
                                                index=index, 
                                                gas_const=gas_const)
    return optimize.fsolve(func, density)[0]

In [48]:
mid_cent_density.value

7.526748814360753e+34

In [51]:
tov.polytropic_density(density_rest=polytropic_density_rest(mid_cent_density.value, n, K_bar),
                       index=n, gas_const=K_bar)

7.526748814360753e+34